In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from timeit import timeit
from IPython.display import display
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from joblib import dump, load
from imblearn.under_sampling import RandomUnderSampler

In [2]:
from compiledataset_reveng import load_dataset, compile_dataset

PATH = "/home/hampus/miun/master_thesis/Datasets"

datasets = {}

# dataset: pd.DataFrame = load_dataset(PATH + "/ORNL", "data_a.csv")
# datasets["ROAD"] = dataset.to_dict("records")

dataset: pd.DataFrame = load_dataset(PATH + "/Survival", "data.csv")
datasets["Survival"] = dataset.to_dict("records")

# dataset: pd.DataFrame = load_dataset(PATH + "/Hisingen", "data.csv")
# datasets["Hisingen"] = dataset.to_dict("records")


df = compile_dataset(datasets)
df.drop(columns=["d0", "d1", "d2", "d3", "d4", "d5", "d6", "d7", "DLC", "t"], inplace=True, errors="ignore")

dataset = None # Release memory, as it isn't used for now
datasets = None

display(df)

,ID,data,data_hex,name,class,dataset,type,Label
0,272,1101000000011100000100000000100100000000000000...,d01c100900000000,FreeDrivingData_20180112_KIA,Soul,Survival,none,0
1,1201,0111100101010011111001011000001000000000000000...,7953e58200000033,FreeDrivingData_20180112_KIA,Soul,Survival,none,0
2,356,0000000000001000000000000000000000000000000000...,000800000000030b,FreeDrivingData_20180112_KIA,Soul,Survival,none,0
3,608,0000010100100010000000000011000011111111100110...,05220030ff99610d,FreeDrivingData_20180112_KIA,Soul,Survival,none,0
4,672,0000001000000000010010001001000011101000000011...,02004890e80f3303,FreeDrivingData_20180112_KIA,Soul,Survival,none,0
...,...,...,...,...,...,...,...,...
192511,1201,0111100101010011111001011000001000000000000000...,7953e58200000033,FreeDrivingData_20180112_KIA,Soul,Survival,none,0
192512,356,0000000000001000000000000000000000000000000000...,0008000000000b03,FreeDrivingData_20180112_KIA,Soul,Survival,none,0
192513,1266,0000000000000000111000000011100000000000000000...,0000e038000000e8,FreeDrivingData_20180112_KIA,Soul,Survival,none,0
192514,1349,1101100000010000000000001000111001000110000000...,d810008e46004a00,FreeDrivingData_20180112_KIA,Soul,Survival,none,0


In [3]:
display(df.loc[df["ID"] == 704, "data"])

204       0011110100000000000000000000000000000000000000...
412       0011110100000000000000000000000000000000000000...
621       0011110100000000000000000000000000000000000000...
829       0011110100000000000000000000000000000000000000...
1037      0011110100000000000000000000000000000000000000...
                                ...                        
191510    0011110100000000000000000000000000000000000000...
191719    0011110100000000000000000000000000000000000000...
191930    0011110100000000000000000000000000000000000000...
192138    0011110100000000000000000000000000000000000000...
192348    0011110100000000000000000000000000000000000000...
Name: data, Length: 921, dtype: object

In [4]:
display(len(np.bincount(df["ID"])))

1681

In [5]:
id_704 = df.loc[df["ID"] == 704, "data"]

l_ids = []

def get_is(l):
    # l_id = []
    bit_int = 0
    for s in l:
        bit_i = int(s, 2)
        bit_int = bit_int | bit_i
        # for i, c in enumerate(s):
        #     if c == "1" and i not in l_id:
        #         l_id.append(i)
    l_ids.append(f"{bit_int:064b}")

df.groupby("ID")["data"].apply(lambda x: get_is(x))


display(l_ids)


['0010100000000000000000010111000000000001000000000010000000010000',
 '0000000000000000000000000000000000000000000000000000000000000000',
 '1111111100000000111111111111111100000000000000000000000000000000',
 '0000000100000000000000000000000000000000000000000000000000000000',
 '0000000000000000000000001111111101000000000000000000000000000000',
 '0000000000100011000000000000000000000000000000000000000000000000',
 '0000000000010111111111100000101100111111000110100011111111111111',
 '0111111110000100010010110000000000000000000000000000000011111111',
 '0101111101111111111111100000101100000000001001110000000000000010',
 '1000011110001111100000001000000000101111000000000000000000000000',
 '1101000000011100000100000000100100000000000000000000000000000000',
 '0000000000000000000000000000000000000000000000000000000000000000',
 '0000000010000000000100001111111100000000111111111111000011111110',
 '0000000000001000000000000000000000000000000000000000111100001111',
 '00001111111110001111111100000000